In [27]:
import pickle
def get_group_idx(group_index):
    # './results/func_code_to_label_{n_clusters}.pkl'
    with open('./results/func_code_to_label_40_balanced.pkl', 'rb') as f:
        data = pickle.load(f)
    all_func_code = []
    for i in range(len(data)):
        if int(data[i][1]) == group_index:
            all_func_code.append(data[i][0])
    return all_func_code

70
70
70
70
70
70
70
70
70
70
70
70
54
70
70
70
70
70
70
59
70
70
70
70
70
70
70
70
70
70
70
46
70
70
70
38
70
70
64
70
70
70
70
36
70
70
70
70
70
70
70
70
70
70
38
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
37
70
70
70
70
70
41
70
70
70
70
70
70
70
70
70
70
70
58
70
70
49
70
70
70
70
70
70
70
70
51
70
70
70
58
70
70
53
70
40
70
69
70
51
70
70
70
70
70
18
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


126

In [ ]:
import json

from sqlalchemy import create_engine, text
import pandas as pd

# 数据库配置
import pickle
with open('./datasets/sql_token.pkl', 'rb') as f:
    DB_URI = pickle.load(f)

engine = create_engine(DB_URI)
DATA_JSON_PATH = './datasets/data.json'

# 加载分类数据
with open(DATA_JSON_PATH) as f:
    group_map = json.load(f)


In [ ]:
for group_idx, (group_name, group_list) in enumerate(group_map.items()):
    for group_data in group_list:
        if group_name == 'stock1' and group_data['industry_code'] in ['270000', '480000']:
            print(group_name, group_data, )
        # break
    # break

In [ ]:
df = {}
for i in range(len(group_map['stock1'])):
    if group_map['stock1'][i]['industry_code'] == '270000':
        print(group_map['stock1'][i])
        df['stock-270000'] = group_map['stock1'][i]['code_list']
df

In [ ]:
import pickle
with open('./datasets/benchmark.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
import pickle
with open('./datasets/benchmark.pkl', 'rb') as f:
    group = pickle.load(f)
group

In [ ]:
pred_value = 'accnav'
sql = text(f"SELECT fund_code, date, nav, accnav, adj_nav FROM b_fund_nav_details_new WHERE fund_code IN :codes AND date BETWEEN :start AND :end ORDER BY date")
df = pd.read_sql_query(
    sql.bindparams(codes=tuple(['013152']), start='2020-07-13', end='2025-03-8'),
    engine
)
df

In [ ]:
df_stamp = pd.to_datetime(df['date'])
df_stamp[1].year, df_stamp[1].month, df_stamp[1].day, df_stamp[1].weekday()

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_excel('./datasets/lottery/pl3_desc.xls', header=1)
timestamps = pd.to_datetime(df['开奖日期'])
timestamps = np.array([[ts.year, ts.month, ts.day, ts.weekday()] for ts in timestamps])
x = np.array(df['号']).reshape(-1, 1)
# timestamps
x = np.concatenate((timestamps, x), axis=-1)
df

In [1]:
import torch
from baselines.Timer.Timer import *
from utils.exp_config import get_config
def get_timer(config):
    # config.patch_len = 96
    # config.d_model = 1024
    # config.d_ff = 2048
    # config.e_layers = 8
    # config.n_heads = 8
    # config.dropout = 0.10
    # config.factor = 21
    # config.output_attention = 1
    # config.activation = 'gelu'
    # config.ckpt_path = 'Timer_forecast_1.0.ckpt'
    timer = Timer(config)
    ckpt_path = 'Timer_forecast_1.0.ckpt'
    sd = torch.load(ckpt_path, weights_only=False, map_location="cpu")["state_dict"]
    sd = {k[6:]: v for k, v in sd.items()}
    timer.load_state_dict(sd, strict=True)
    return timer 
config = get_config('FinancialConfig')
timer = get_timer(config)

<module 'module.name' from '/home/rtx4090/code/python/当前任务/TimeSeries/configs/FinancialConfig.py'> FinancialConfig


In [15]:
x = torch.rand(10, 3, 96)
print(x.shape)
x_shape = x.shape
x = timer.patch_embedding(x)[0].reshape((x_shape[0], x_shape[1], -1))
print(x.shape)
x, _= timer.decoder(x)
print(x.shape)
x = timer.proj(x)
print(x.shape)

torch.Size([10, 3, 96])
torch.Size([10, 3, 1024])
torch.Size([10, 3, 1024])
torch.Size([10, 3, 96])


In [18]:
import einops
x = torch.rand(10, 96, 68, 3)
for i in range(x.shape[-2]):
    now = einops.rearrange(x[:, :, i, :], 'bs seq d -> bs d seq')
    now_shape = now.shape
    now = timer.patch_embedding(now)[0].reshape((now_shape[0], now_shape[1], -1))
    now, _= timer.decoder(now)
    now = timer.proj(now)
    x[:, :, i, :] = einops.rearrange(now, 'bs d seq -> bs seq d')

In [4]:
from data_provider.exp_dataloader import DataModule
from exp.exp_model import Model
from utils.exp_logger import Logger
from utils.exp_metrics_plotter import MetricsPlotter
from run_train import get_experiment_name
from utils.utils import set_settings
# Experiment Settings, logger, plotter
from utils.exp_config import get_config
config = get_config('FinancialConfig')
config.multi_dataset = True
set_settings(config)
log_filename, exper_detail = get_experiment_name(config)
plotter = MetricsPlotter(log_filename, config)
log = Logger(log_filename, exper_detail, plotter, config)
datamodule = DataModule(config)
model = Model(config)

012497 -- len = 867, now min_length = 704
012696 -- len = 893, now min_length = 704
012697 -- len = 893, now min_length = 704
|2025-06-05 21:38:35| Train_length : 467 Valid_length : 81 Test_length : 81


In [ ]:
import torch
import torch.nn as nn

# 假设输入
bs, seq_len, channels, dim = 16, 48, 33, 64
x_enc = torch.randn(bs, seq_len, channels, dim)

# 定义 attention 层（无 batch_first 参数）
attn_channel = nn.MultiheadAttention(embed_dim=dim, num_heads=8)  # expects (seq_len, batch, dim)
attn_time = nn.MultiheadAttention(embed_dim=dim, num_heads=8)

# ===== 1. 跨通道 attention =====
# 原始 x_enc: (bs, 48, 33, 64)
# 调整为 (33, bs*48, 64)
x_enc_reshaped = x_enc.permute(2, 0, 1, 3).reshape(channels, bs * seq_len, dim)

# 注意力：通道之间的 self-attention
x_channel_attn, _ = attn_channel(x_enc_reshaped, x_enc_reshaped, x_enc_reshaped)  # (33, bs*48, 64)

# 还原为 (bs, 48, 33, 64)
x_channel_attn = x_channel_attn.reshape(channels, bs, seq_len, dim).permute(1, 2, 0, 3)

# ===== 2. 跨时间 attention =====
# 调整为 (48, bs*33, 64)
x_time_input = x_channel_attn.permute(1, 0, 2, 3).reshape(seq_len, bs * channels, dim)

# 注意力：时间步之间的 self-attention
x_time_attn, _ = attn_time(x_time_input, x_time_input, x_time_input)  # (48, bs*33, 64)

# 还原为 (bs, 48, 33, 64)
x_time_attn = x_time_attn.reshape(seq_len, bs, channels, dim).permute(1, 0, 2, 3)

# 最终输出
print(x_time_attn.shape)  # torch.Size([16, 48, 33, 64])


In [26]:
from modules.backbone import Backbone
from run_train import *

from utils.exp_config import get_config
config = get_config()
# datamodule = DataModule(config)
# model = Model(datamodule, config)
model = Backbone(3, config)

In [27]:
bs, seq_len, channels, dim = 1, 48, 33, 3
random_inputs = torch.rand(bs, seq_len, channels, dim)
y = model(random_inputs, None, None)
# [1, 48, 32, 3]

In [28]:
bs, seq_len, channels, dim = 1, 48, 1, 3
random_inputs = torch.rand(bs, seq_len, channels, dim)
y = model(random_inputs, None, None)
# [1, 48, 32, 3]

In [17]:
bs, seq_len, channels, dim = 1, 48, 16, 3
random_inputs = torch.rand(bs, seq_len, channels, dim)
y = model(random_inputs, None, None)
# [1, 48, 32, 3]

In [1]:
import torch

x = torch.arange(2*3*4*10).reshape(2, 3, 4, 10)
patch_len = 4
stride = 2

x_unfolded = x.unfold(dimension=-1, size=patch_len, step=stride)
print(x_unfolded.shape)

torch.Size([2, 3, 4, 4, 4])


In [10]:
import os

# 读取所有文件名
all_files = os.listdir('results/ours/20250619/log')

# 提取每个文件中 _Multi_ 与 .md 之间的数字
existing_ids = set()
for filename in all_files:
    try:
        num = int(filename.split('_Multi_')[1].split('.md')[0])
        existing_ids.add(num)
    except (IndexError, ValueError):
        continue

# 检查 1-130 中缺失的编号
missing_ids = [i for i in range(0, 125) if i not in existing_ids]

print("缺失的编号：", missing_ids)

缺失的编号： [23, 28, 30, 36, 37, 55, 64, 66, 72, 79, 120]
